In [2]:
from __future__ import unicode_literals

import json

from hazm import *


def read_file():
    documents_title = []
    documents_content = []
    documents_url = []
    with open('../Phase_1/assets/IR_data_news_12k.json', encoding='UTF-8') as f:
        data = json.load(f)
        for i in data:
            documents_title.append(data[i]["title"])
            documents_content.append(data[i]["content"])
            documents_url.append(data[i]['url'])
    return documents_url, documents_title, documents_content


docs_url, docs_title, docs_content = read_file()
normalizer = Normalizer()

In [3]:
# coding: utf8
from os import path
import codecs
from hazm.Normalizer import Normalizer
from Phase_1.src.utils import data_path

default_stop_words = path.join(data_path, 'stopwords.dat')


class StopWord:
    """ Class for remove stop words

         >>> StopWord().clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران', 'کی']
         >>> StopWord(normal=True).clean(["در","تهران","کی","بودی؟"])
         ['بودی؟', 'تهران']

         """

    def __init__(self, file_path=default_stop_words, normal=False):
        self.file_path = file_path
        self.normal = normal
        self.normalizer = Normalizer().normalize
        self.stop_words = self.init(file_path, normal)

    def init(self, file_path, normal):
        if not normal:
            return set(
                line.strip("\r\n") for line in codecs.open(file_path, "r", encoding="utf-8").readlines())
        else:
            return set(
                self.normalizer(line.strip("\r\n")) for line in
                codecs.open(file_path, "r", encoding="utf-8").readlines())

    def set_normalizer(self, func):
        self.normalizer = func
        self.stop_words = self.init(self.file_path, self.normal)

    def __getitem__(self, item):
        return item in self.stop_words

    def __str__(self):
        return str(self.stop_words)

    def clean(self, iterable_of_strings, return_generator=False):
        if return_generator:
            return filter(lambda item: not self[item], iterable_of_strings)
        else:
            return list(filter(lambda item: not self[item], iterable_of_strings))


In [4]:
class Document:
    def __init__(self, content, url, title):
        self.content = content
        self.url = url
        self.title = title


In [5]:
def preprocess_content(content):
    str_empty = ' '
    stemmer = Stemmer()
    content = normalizer.normalize(content)
    content = str_empty.join([stemmer.stem(word) for word in content.split()])
    content = word_tokenize(content)
    content = StopWord(normal=False).clean(content)
    return str_empty.join(content)


def process_positions(preprocessed_content):
    positions = {}
    for position_of_word, word in enumerate(preprocessed_content.split()):
        if word and word not in positions:
            positions[word] = []
        positions[word].append(position_of_word)
    return positions


def generate_index(positions, title):
    return {"number of occurrences in document": len(positions),
            "positions": positions,
            "title of document": title}


class PositionalIndex:
    def __init__(self, documents_url, documents_title, documents_content):
        self.Documents = [Document(documents_content[i], documents_url[i], documents_title[i]) for i in
                          range(len(documents_url))]
        self.positional_index_structure = {}
        self.url_to_information = {}
        self.build_positional_index()
        self.build_url_to_information_dict()

    def build_positional_index(self):
        for i, document in enumerate(self.Documents):
            preprocessed_content = preprocess_content(document.content)
            processed_content = process_positions(preprocessed_content)
            title = document.title
            url = document.url
            for word, positions in processed_content.items():
                self.add_index(word, positions, title, url)

    def add_index(self, word, positions, title, url):
        index_to_add = generate_index(positions, title)
        if word not in self.positional_index_structure:
            self.positional_index_structure[word] = {"total occurrences": 0,
                                                     "indexes": {}}
        self.positional_index_structure[word]["total occurrences"] += index_to_add[
            "number of occurrences in document"]
        self.positional_index_structure[word]["indexes"][url] = index_to_add

    def build_url_to_information_dict(self):
        for document in self.Documents:
            self.url_to_information[document.url] = {"title": document.title,
                                                     "content": document.content}


In [6]:
pos_index = PositionalIndex(documents_url=docs_url, documents_title=docs_title, documents_content=docs_content)

In [18]:
def docID(number, lst_urls):
    return lst_urls[number]


def position(plist):
    return plist['positions']


def pos_intersect(p1, p2, k):
    answer = {}  # answer <- ()
    len1 = len(p1)
    len2 = len(p2)
    i = j = 0
    lst_urls1 = sorted(list(p1.keys()))
    lst_urls2 = sorted(list(p2.keys()))
    while i != len1 and j != len2:
        key = docID(i, lst_urls1)
        key2 = docID(j, lst_urls2)
        if key == key2:
            l = []
            pp1 = position(p1[key])
            pp2 = position(p2[key])

            plen1 = len(pp1)
            plen2 = len(pp2)
            ii = jj = 0
            while ii != plen1:
                while jj != plen2:
                    if abs(pp1[ii] - pp2[jj]) <= k:
                        l.append(pp2[jj])
                    elif pp2[jj] > pp1[ii]:
                        break
                    jj += 1
                # l.sort()
                while l != [] and abs(l[0] - pp1[ii]) > k:
                    l.remove(l[0])
                for ps in l:
                    if key in answer:
                        # answer[key]['positions'].extend([pp1[ii], ps])
                        answer[key]['positions'].extend([max(pp1[ii], ps)])
                    else:
                        answer[key] = {'positions': [max(pp1[ii], ps)]}
                ii += 1
            i += 1
            j += 1
            if key in answer:
                answer[key]['positions'] = list(set(answer[key]['positions']))
        elif key < key2:
            i += 1
        else:
            j += 1
    return answer


def calc_string(string):
    state = False
    i = 0
    while i < len(string):
        ch = string[i]
        if ch == "\"":
            if state:
                string = string[:i] + '>' + string[i + 1:]
            else:
                string = string[:i] + '<' + string[i + 1:]
            state = 1 ^ state
        elif ch == '!':
            string = string[:i - 1] + ' _NOT_ ' + string[i + 1:]
        i += 1
    return string


def preprocess_query(query):
    query = calc_string(query)
    return preprocess_pipeline(query)


def preprocess_pipeline(query):
    str_empty = ' '
    stemmer = Stemmer()
    query = normalizer.normalize(query)
    query = str_empty.join([stemmer.stem(word) for word in query.split()])
    query = word_tokenize(query)
    query = StopWord(normal=False).clean(query)
    query = str_empty.join(query)
    return query


class QueryHandler:
    def __init__(self, positional_index):
        self.positional_index = positional_index

    def answer_query(self, query):
        query = preprocess_query(query)
        not_queries = []
        and_queries = []
        empty_str = ' '
        is_not = False
        is_consecutive = False
        consecutive = []

        i = 0
        while i < len(query):
            next_occur = query.find(empty_str, i)
            substr = query[i:next_occur]
            if next_occur == -1:
                substr = query[i:]
            if substr == '_NOT_':
                is_not = True
                i = next_occur + 1
                continue
            elif is_not:
                not_queries.append(self.get_result([substr]))
                is_not = False
                i = next_occur + 1
                if next_occur == -1:
                    break
                continue
            elif substr[0] == '<':
                word = preprocess_pipeline(substr[1:])
                if word:
                    consecutive.append(word)
                i = next_occur + 1
                is_consecutive = True
                continue
            elif substr[-1] == '>':
                word = preprocess_pipeline(substr[:-1])
                if word:
                    consecutive.append(word)
                and_queries.append(self.get_result(consecutive))
                i = next_occur + 1
                consecutive.clear()
                is_consecutive = False
                if next_occur == -1:
                    break
                continue
            elif is_consecutive:
                consecutive.append(substr)
                i = next_occur + 1
                continue
            and_queries.append(self.get_result([substr]))
            i = next_occur + 1
            if next_occur == -1:
                break
        and_results = self.handle_queries(and_queries)
        not_results = self.handle_queries(not_queries)
        query_result = self.remove_by_sub_queries(and_results, not_results)
        get_score = lambda x: x['score']
        return sorted(query_result, key=get_score, reverse=True)

    def remove_by_sub_queries(self, in_results, out_results):
        answer = []
        for in_item_url in in_results:
            if in_item_url in out_results:
                continue
            index = {'title': self.positional_index.url_to_information[in_item_url]['title'],
                     'score': len(in_results[in_item_url]['positions']),
                     'url': in_item_url,
                     'content': self.positional_index.url_to_information[in_item_url]['content']}
            answer.append(index)
        return answer

    @staticmethod
    def handle_queries(queries):
        if not queries:
            return {}
        temporary_result = queries[0]
        for item_to_intersect in queries[1:]:
            if temporary_result:
                temporary_result = pos_intersect(temporary_result, item_to_intersect, 1000000)
            else:
                temporary_result = item_to_intersect
        return temporary_result

    def get_result(self, words):
        if words[0] not in self.positional_index.positional_index_structure:
            return {}
        temp_result = self.positional_index.positional_index_structure[words[0]]['indexes']
        for word in words[1:]:
            if word not in self.positional_index.positional_index_structure:
                temp_result = {}
                break
            indexes_of_word = self.positional_index.positional_index_structure[word]['indexes']
            if temp_result:
                temp_result = pos_intersect(temp_result, indexes_of_word, 1)
            else:
                temp_result = indexes_of_word
        return temp_result


query_handler = QueryHandler(positional_index=pos_index)
print(query_handler.answer_query('تحریم هسته ای'))

[{'title': 'اهرم\u200cسازی از افغانستان در برجام/ نقطه عزیمت آمریکا در مذاکرات جامع با ایران چیست؟', 'score': 38, 'url': 'https://www.farsnews.ir/news/14000803000676/اهرم\u200cسازی-از-افغانستان-در-برجام-نقطه-عزیمت-آمریکا-در-مذاکرات-جامع-با', 'content': '\nخبرگزاری فارس؛ محسن صمیمی، مرتضی علیزاده:\xa0یکی ـ دو ماه پایانی دولت روحانی، اروپا و آمریکا تقریبا از ادامه مذاکرات وین دست کشیدند و منتظر ماندند تا دولت سیّد ابراهیم رئیسی کار خود را آغاز کند. آنها می دانستند که رویکرد سیاست خارجی دولت جدید تفاوت فاحشی با رویکرد دولت قبل دارد و نمی توان با دست فرمان سابق با ایران تعامل و یا به عبارتی منازعه دیپلماتیک کرد.\xa0 دولت جدید که روی کار آمد رسما گفت که پیش شرط ورود ایران به مذاکرات، انجام تعهدات برجامی آمریکاست و به عبارتی باید تحریم ها لغو شوند. آمریکایی ها هم به صراحت گفتند که خبری از لغو تحریم نیست. در این میان آژانس بین المللی انرژی اتمی با مدیریت رافائل گروسی نقش فعالی را ایفا می کند و سفرهایی را به ایران ترتیب داده است. اروپا نیز نماینده خود را به ایران فرستاد و کنش\u200cهای سیاسی و 

In [8]:
def print_results(query_string, number_to_prints=10):
    for single_result in query_handler.answer_query(query_string)[:number_to_prints]:
        print(f"score: {single_result['score']},\n"
              f'content: {single_result["content"]}')

In [9]:
print_results('تحریم های آمریکا علیه ایران')

score: 133,
content: 
به گزارش خبرگزاری فارس، مرکز پژوهش‌های مجلس شورای اسلامی در گزارشی با عنوان «مذاکرات وین: تقابل اراده‌ها و جنگ روایت‌ها» به موضوع مذاکرات اخیر ایران با 1+4 پرداخت.  در بخش چکیده این گزارش آمده است: انگیزه اصلی جمهوری اسلامی ایران برای ورود به مذاکرات وین، تحقق هدف مندرج در بند «29» برجام یعنی «عادی‌سازی روابط اقتصادی و تجاری» بود. با این‌حال، نتایج 6 دور اول گفتگوهای وین به‌دلیل عدم اراده آمریکا برای لغو مؤثر همه تحریم‌ها و ارائه تضمین به‌لحاظ اقتصادی، منافع ملی جمهوری اسلامی ایران را تأمین نمی‌کرد.  به‌دلیل عدم انتفاع اقتصادی، تجدیدنظر درباره پیش‌نویس 6 دور قبلی مذاکرات یک ضرورت برای جمهوری اسلامی ایران محسوب می‌شد، ولی آمریکا ـ حتی پیش از تدوین متن جدید در تهران ـ با ایجاد کمپین روانی خواسته ایران را «زیاده‌خواهانه و فرابرجامی» معرفی کرد. درحالی که، طبق برجام و قطعنامه 2231، آمریکا علاوه‌بر رفع تحریم‌های مقرر در ضمیمه 2 برجام باید از اتخاذ اقداماتی که مانع از عادی‌سازی روابط اقتصادی و تجاری با ایران می‌شود، خودداری نماید. آمریکا برای تغییر محاسبه جمهوری اسلامی ا

In [10]:
print_results('تحریم های آمریکا ! ایران')

score: 9,
content: 
به گزارش خبرنگار سیاسی خبرگزاری فارس، آیت‌الله سیداحمد خاتمی امام جمعه موقت تهران، امروز در خطبه‌های اول نماز جمعه که در مصلای امام خمینی (ره) اقامه شد، در ادامه سلسله مباحث سبک زندگی اظهار داشت: آیین‌نامه زندگی خدایی در قرآن کریم و در آیه 24 سوره انفال آمده است که بر اساس آن، هر آنچه را که خدا و پیامبر اکرم (ص) دستور داده‌اند، باید عمل شود. وی ادامه داد: رسالت دین فقط آبادی آخرت نیست، بلکه دین به دنبال آبادی دنیا و آخرت است؛ کسانی که بخاطر دنیا، آخرت را فراموش می‌کنند و کسانی که بخاطر آخرت، دنیای خود را فراموش می‌کنند، هیچکدام مطلوب دین نیستند. خطیب نماز جمعه تهران تصریح کرد: این نقطه، مقابل تفکرات انحرافی سکولاریزم مذهبی است که می‌گویند کار دین، فقط آبادی آخرت است و دنیا را به اهل دنیا بسپارید. خاتمی با اشاره به خداباوری و عمل صالح به عنوان ستون فقرات تفکر دینی گفت: باید ایمان‌مدار باشیم،‌ نه ایمان‌شعار. حیات واقعی در سایه دین و عمل به دستورات خداوند و پیامبر (ص) است و آنهایی که این زندگی را ندارند، زندگی حیوانی دارند. وی همچنین با اشاره به اهمیت مسئله «جهاد تبیین

In [11]:
print_results('"مردم ایران"')

score: 7,
content: 
خبرگزاری فارس - مهدی امامقلی: در ماجرای بنزینی آبان 98، آمریکا و چند کشور اروپایی که در کارزاری گسترده علیه جمهوری اسلامی ایران فعال شده بودند و علناً و رسماً از آشوبگران حمایت می‌کردند با نقش‌آفرینی ملت آگاه ایران و مدیریت نظام اسلامی، شکست سنگینی را متحمل شدند. حالا تقریباً دو آبان گذشته یکی آبان 99 و یکی هم آبان 1400 که روزهای پایانی خود را سپری می‌کند. ایران همچنان در آرامش است و دشمن همچنان در عصبانیت چنانکه در طول 42 سال گذشته این‌چنین بوده است و خواهد بود. بخش فارسی بنگاه سخن‌پراکنی BBC که آن ایام، اخبار و گزارش‌های خود را بر اعتراضات در ایران متمرکز کرده بود در ادامه شیطنت‌ها و خباثت‌های رسانه‌ای خود، شب گذشته با انتشار گزارش جعلی و غیرواقعی از آبان 98 سعی کرد با تداعی آن روزها بار دیگر بر مسئله «اعتراض» در ایران بنزین بپاشد و با مقصرنمایی نظام جمهوری اسلامی، نوع مدیریت رهبر انقلاب در مواجهه با اعتراضات 98 را زیر سؤال ببرد. رهبر انقلاب و جداکردن صف «مردم» از «اشرار» 1. شبکه ملکه انگلیس در گزارش اخیر خود سعی کرده است که القا کند مواجهه رهبر انقلاب با حرکت‌های

In [12]:
print_results('"خبرگزاری فارس"')


score: 18,
content: 
خبرگزاری فارس در ۲۲ بهمن سال ۱۳۸۱ و در شرایط غربت انقلاب در فضای رسانه‌ای کشور متولد شد و خیلی زود توانست با شعار «فارس فرزند ایران است» جایگاه خود را به عنوان پرمخاطب‌ترین خبرگزاری کشور و یکی از رسانه‌های اثرگذار جریان مقاومت تثبیت کند. فارس تلاش می‌کند گامی در جهت روشن کردن حقایق داشته باشد و معتقد است که سرانجام «حقیقت روشن می‌شود». بنابراین هویت بصری خود را مبتنی بر این شعار و بر مبنای عناصر کلیدی اعتماد، انقلاب و امید و مبتنی بر ماموریت‌های جدید بازطراحی کرده است. صاحب امتیاز خبرگزاری فارس، موسسه فرهنگی و رسانه‌ای فارس است و هزینه‌های این خبرگزاری از طریق درآمدهای آموزشی، پژوهشی و تبلیغاتی و نیز با حمایت نهادهای انقلابی تامین می‌شود. در سال جاری مطابق با آمار گوگل، خبرگزاری فارس ۲۵ میلیون مخاطب منحصر به فرد داشته است که این تعداد مجموعا صدها میلیون بار اخبار خبرگزاری فارس را مشاهده کرده‌اند. این میزان مخاطب در حالی است که از ۲ سال قبل دامنه اصلی خبرگزاری فارس تحت تحریم رژیم آمریکا قرار گرفت و در اقدامی بی‌سابقه مسدود شد. با این حال در حال حاضر حدود ۱۱ درصد مخا

In [13]:
print_results('"کنگره ضدتروریست "')

score: 1,
content: 
به گزارش خبرنگار سیاسی خبرگزاری فارس، به نظر می‌رسد مذاکرات وین به روزهای آخر خود نزدیک شده است و براساس اعلام برخی منابع تا رسیدن به توافق راه زیادی باقی نمانده و تنها نیاز است آمریکا به آنچه در برجام برای برداشتن تحریم‌ها وعده داده  عمل کند و تضمین معتبری هم برای انجام این تعهدات داده شود. براساس اعلام مسئولان سیاست خارجه در کنار دریافت تضمین‌ معتبر، برداشتن همه تحریم‌ها از نکات و مطالبات اصلی ایران در مذاکرات است. اما به نظر می‌رسد آمریکا در مقابل عمل به این تعهد برجامی مقاومت می‌کند. * تحریم‌های آمریکا علیه ایران آمریکا پس از تسخیر لانه‌ جاسوسی آمریکا در تهران، «کارتر» در تاریخ 8 نوامبر 1979م. با استناد به قانون‌ «کنترل صدور تسلیحات نظامی»، کشتی حامل‌ لوازم یدکی نظامی متعلق به ایران را توقیف کرد. ارزش این لوازم 300 میلیون دلار بود. با اوج‌گیری کشمکش‌های سیاسی بر سر مسئله‌ تصرف سفارت، دولت موقت، اعلام کرد تمام دارایی‌های‌ خود را از بانک‌های آمریکا خارج خواهد کرد. کارتر با لحاظ این احتمال، در کشور شرایط اضطراری اعلام کرد و با استناد به قانون شرایط اضطراری اقتصاد ب

In [16]:
print_results('"دانشگاه صنعتی ا میرکبیر" رهبر ! خدا')

score: 20,
content: 
گروه سیاسی خبرگزاری فارس: با توجه به نزدیک شدن به یوم‌الله نهم دی، در این خبرگزاری به بازخوانی و روایت حوادث رخ داده در سال 88، از زمان انتخابات ریاست جمهوری تا حماسه مردمی 9 دی که مهر پایانی بر فتنه 8 ماهه در کشور بود، پرداختیم که در ادامه آمده است: خردادماه،‌ ماه پر حادثه ایرانی‌ها شروع شده و مشخص است که مردم باید از بین چه کسانی رئیس جمهور آینده خود را برگزینند. محمود احمدی‌نژاد که رئیس جمهور مستقر است از یک سو، و میرحسین موسوی که پس از سال‌ها از پس پرده کارهای هنری‌اش درآمده و عزم بازگشت به سیاست کرده است، از سویی دیگر بارزترین کاندیداها هستند. البته مهدی کروبی و محسن رضایی هم هستند ولی میدان بازی حاکی از این است که اصلا‌ح‌طلبان روی موسوی قمار می‌کنند. جدال وعده‌ها وعده‌های موسوی از جنس وعده‌های همیشگی اصلاحات است و سویه‌های جامعه مدنی و آزادی‌سیاسی دارد. هرچند بدون ارائه راهکار از بیکاری و فقر و مشکلاتی از این دست هم می‌نالد. میر حسین موسوی در نخستین سخنان انتخاباتی اش دوم خردادماه می‌گوید که «طرح امینت اجتماعی نمی تواند تامین کننده نظرها و اهداف حکومت ایران ب